In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
img = cv2.imread("image1.jpg")
mp_facemsh = mp.solutions.face_mesh
face_mesh = mp_facemsh.FaceMesh(static_image_mode = True,refine_landmarks=True)
img.shape

In [ ]:
results = face_mesh.process(img)

In [ ]:
landmarks = results.multi_face_landmarks[0]

In [ ]:
face_oval = mp_facemsh.FACEMESH_FACE_OVAL
df = pd.DataFrame(list(face_oval),columns=["p1","p2"])

In [ ]:
routes_idx = []

p1 = df.iloc[0]["p1"]
p2 = df.iloc[0]["p2"]
for i in range(0,df.shape[0]):
    obj = df[df["p1"]==p2]
    p1 = obj["p1"].values[0]
    p2 = obj["p2"].values[0]
    current_route = []
    current_route.append(p1)
    current_route.append(p2)
    routes_idx.append(current_route)

In [ ]:
routes = []
 
for source_idx, target_idx in routes_idx:
     
    source = landmarks.landmark[source_idx]
    target = landmarks.landmark[target_idx]
         
    relative_source = (int(img.shape[1] * source.x), int(img.shape[0] * source.y))
    relative_target = (int(img.shape[1] * target.x), int(img.shape[0] * target.y))
 
    #cv2.line(img, relative_source, relative_target, (255, 255, 255), thickness = 2)
     
    routes.append(relative_source)
    # routes.append(relative_target)
print(len(routes))

In [ ]:
mask = np.zeros((img.shape[0],img.shape[1]))
mask = cv2.fillConvexPoly(mask,np.array(routes),1)
mask = mask.astype(bool)
out = np.zeros_like(img)
out[mask] = img[mask]


In [ ]:
plt.imshow(out[:,:,::-1])

In [ ]:
# Delaunay Triangulation
rect = cv2.boundingRect(np.array(routes))
# (x,y,w,h)  =rect
# cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0))
subdiv = cv2.Subdiv2D(rect)
subdiv.insert(routes)
triangles = subdiv.getTriangleList()
triangles = np.array(triangles, dtype = int)
for t in triangles:
    pt1 = (t[0],t[1])
    pt2 = (t[2],t[3])
    pt3 = (t[4],t[5])

    # cv2.line(img, pt1, pt2, (0, 0, 255), 1, cv2.LINE_AA, 0)
    # cv2.line(img, pt2, pt3, (0, 0, 255), 1, cv2.LINE_AA, 0)
    # cv2.line(img, pt3, pt1, (0, 0, 255), 1, cv2.LINE_AA, 0)


plt.imshow(img[:,:,::-1])

In [ ]:
def extract_index(nparray):
    return nparray[0][0]
routes = np.array(routes,np.int32)

triangle_indexes = []
for t in triangles:
    pt1 = (t[0],t[1])
    pt2 = (t[2],t[3])
    pt3 = (t[4],t[5])

    index_p1 = np.where((routes == pt1).all(axis=1))
    index_p1 = extract_index(index_p1)

    index_p2 = np.where((routes == pt2).all(axis=1))
    index_p2 = extract_index(index_p2)

    index_p3 = np.where((routes == pt3).all(axis=1))
    index_p3 = extract_index(index_p3)
    
    

    if index_p1 is not None and index_p2 is not None and index_p3 is not None:
        triangle = [index_p1,index_p2,index_p3]
        triangle_indexes.append(triangle)


In [ ]:
img2 = cv2.imread("image2.jpeg")
results2 = face_mesh.process(img2)
landmarks2 = results2.multi_face_landmarks[0]

routes2 = []
 
for source_idx, target_idx in routes_idx:
     
    source = landmarks2.landmark[source_idx]
    target = landmarks2.landmark[target_idx]
         
    relative_source = (int(img2.shape[1] * source.x), int(img2.shape[0] * source.y))
    relative_target = (int(img2.shape[1] * target.x), int(img2.shape[0] * target.y))
 
    #cv2.line(img, relative_source, relative_target, (255, 255, 255), thickness = 2)
     
    routes2.append(relative_source)

    # routes.append(relative_target)

points2 = np.array(routes2,np.int32)
convexhull2 = cv2.convexHull(points2)
# 2nd image Traingulation

for triangle_index in triangle_indexes:
    pt1 = routes2[triangle_index[0]]
    pt2 = routes2[triangle_index[1]]
    pt3 = routes2[triangle_index[2]]

    # cv2.line(img2, pt1, pt2, (0, 0, 255), 1, cv2.LINE_AA, 0)
    # cv2.line(img2, pt2, pt3, (0, 0, 255), 1, cv2.LINE_AA, 0)
    # cv2.line(img2, pt3, pt1, (0, 0, 255), 1, cv2.LINE_AA, 0)

plt.imshow(img2[:,:,::-1])
    


In [ ]:
img2_new_face = np.zeros_like(img2)

In [ ]:
img2_new_face = np.zeros_like(img2,dtype=np.uint8)
count = 0
for triangle_index in triangle_indexes:
    tr1_pt1 = routes[triangle_index[0]]
    tr1_pt2 = routes[triangle_index[1]]
    tr1_pt3 = routes[triangle_index[2]]

    triangle1 = np.array([tr1_pt1, tr1_pt2, tr1_pt3], np.int32)
    rect1 = cv2.boundingRect(triangle1)
    (x1, y1, w1, h1) = rect1

    cropped_triangle1 = img[y1:y1 + h1, x1:x1 + w1]
    cropped_triangle1_mask = np.zeros((h1, w1), np.uint8)

    points1 = np.array([[tr1_pt1[0] - x1, tr1_pt1[1] - y1],
                        [tr1_pt2[0] - x1, tr1_pt2[1] - y1],
                        [tr1_pt3[0] - x1, tr1_pt3[1] - y1]], int)

    cv2.fillConvexPoly(cropped_triangle1_mask, points1, 255)
    cropped_triangle1 = cv2.bitwise_and(cropped_triangle1, cropped_triangle1, mask=cropped_triangle1_mask)

    tr2_pt1 = routes2[triangle_index[0]]
    tr2_pt2 = routes2[triangle_index[1]]
    tr2_pt3 = routes2[triangle_index[2]]

    triangle2 = np.array([tr2_pt1, tr2_pt2, tr2_pt3], np.int32)
    rect2 = cv2.boundingRect(triangle2)
    (x2, y2, w2, h2) = rect2

    cropped_triangle2_mask = np.zeros((h2, w2), np.uint8)

    points2 = np.array([[tr2_pt1[0] - x2, tr2_pt1[1] - y2],
                        [tr2_pt2[0] - x2, tr2_pt2[1] - y2],
                        [tr2_pt3[0] - x2, tr2_pt3[1] - y2]], int)

    cv2.fillConvexPoly(cropped_triangle2_mask, points2, 255)

    # Warp Triangle
    points1 = np.float32(points1)
    points2 = np.float32(points2)
    M = cv2.getAffineTransform(points1, points2)
    warped_triangle = cv2.warpAffine(cropped_triangle1, M, (w2, h2))
    #smooth edges
    warped_triangle = cv2.GaussianBlur(warped_triangle, (5,5), sigmaX=0.2, sigmaY=1)
    # _, mask_triangles_designed = cv2.threshold(img2_new_face[y2:y2+h2,x2:x2+w2], 1, 255, cv2.THRESH_BINARY_INV)
    # warped_triangle = cv2.bitwise_and(warped_triangle, warped_triangle, mask=mask_triangles_designed)
    # plt.imshow(warped_triangle[:,:,::-1])
    # plt.show()
    

    # Blend the warped triangle into the new face
    mask_triangle_area = cv2.bitwise_and(warped_triangle, warped_triangle, mask=cropped_triangle2_mask)
    img2_new_face[y2:y2 + h2, x2:x2 + w2] = cv2.add(img2_new_face[y2:y2 + h2, x2:x2 + w2], mask_triangle_area)
    count = count + 1
    # if count == 5:

    #     break

# Final steps
img2_new_face_gray = cv2.cvtColor(img2_new_face, cv2.COLOR_BGR2GRAY)
_, background = cv2.threshold(img2_new_face_gray, 1, 255, cv2.THRESH_BINARY_INV)
background = cv2.bitwise_and(img2, img2, mask=background)
final_result = cv2.add(img2_new_face, background)

plt.imshow(final_result[:, :, ::-1])
plt.show()


In [ ]:
img2_grey = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
img2_face_mask = np.zeros_like(img2_grey)
img2_head_mask = cv2.fillConvexPoly(img2_face_mask, convexhull2, 255)
img2_face_mask = cv2.bitwise_not(img2_head_mask)


img2_head_noface = cv2.bitwise_and(img2, img2, mask=img2_face_mask)
result = cv2.add(img2_head_noface, img2_new_face)

(x, y, w, h) = cv2.boundingRect(convexhull2)
center_face2 = (int((x + x + w) / 2), int((y + y + h) / 2))

seamlessclone = cv2.seamlessClone(result, img2, img2_head_mask, center_face2, cv2.NORMAL_CLONE)

plt.imshow(seamlessclone[:,:,::-1])
# plt.imshow(img2_face_mask)